In [1]:
import re
import pandas as pd

In [3]:
plt.rcParams['font.family']='AppleGothic'
plt.rcParams['axes.unicode_minus']=False

In [2]:
구글리뷰=pd.read_csv('구글리뷰_백업.csv')
리뷰=pd.read_csv("한식리뷰_카카오.csv")

NameError: name 'pd' is not defined

In [6]:
구글리뷰.drop_duplicates(inplace=True)
구글리뷰['label']=구글리뷰.별점.map({' 별표 5개 ':0, ' 별표 3개 ':2, ' 별표 4개 ':0, ' 별표 1개 ':1, ' 별표 2개 ':1})
구글리뷰['리뷰']=구글리뷰['리뷰'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣0-9\s]", "", regex=True)
구글리뷰['리뷰'].replace('',np.nan,inplace=True)
구글리뷰.dropna(inplace=True)
리뷰.drop_duplicates(inplace=True)
리뷰['label']=리뷰.별점.map({'5점':0,'4점':0,'3점':2,'2점':1,'1점':1})
리뷰['리뷰']=리뷰['리뷰'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣0-9\s]", "", regex=True)
리뷰['리뷰'].replace('',np.nan,inplace=True)
리뷰.dropna(inplace=True)
리뷰=리뷰.append(구글리뷰)
리뷰=리뷰.reset_index().drop('index',axis=1)
for i in 리뷰[리뷰['리뷰'].str.contains('번역 제공')].index:
    리뷰.loc[i,'리뷰']=리뷰.loc[i,'리뷰'].replace('번역 제공','').replace('원문','')
for i in 리뷰[리뷰['리뷰'].str.contains('\n')].index:
    리뷰.loc[i,'리뷰']=리뷰.loc[i,'리뷰'].replace('\n','')
for i in 리뷰[리뷰['리뷰'].str.contains('\u2028')].index:
    리뷰.loc[i,'리뷰']=리뷰.loc[i,'리뷰'].replace('\u2028','')

In [11]:
checked_sents=[spell_checker.check(i).checked for i in 리뷰['리뷰']]

In [13]:
checked_sents[0]

'가보면 내 맘 알지'

In [14]:
리뷰['리뷰']=checked_sents

In [16]:
리뷰.isna().sum()

리뷰       0
별점       0
label    0
dtype: int64

In [18]:
리뷰.to_csv('review.csv',index=False)

In [23]:
네이버=pd.read_csv('naver_review.csv')
네이버.label.unique()

array([0.5, 5. , 4. , 4.5, 1.5, 2.5, 3.5, 2. , 3. , 1. ])

In [116]:
네이버_fil=네이버[(네이버['label']==0.5)|(네이버['label']==1.)|(네이버['label']==2.)|(네이버['label']==1.5)|(네이버['label']==2.5)]
네이버_fil_3=네이버[네이버['label']==3]

In [117]:
for i in 네이버_fil.query('["굿","굳","좋아요","good","ㅇ","Good","GOOD"] in text').index:
    네이버_fil.drop(i,inplace=True)
for i in 네이버_fil_3.query('["굿","굳","좋아요","good","ㅇ","Good","GOOD"] in text').index:
    네이버_fil_3.drop(i,inplace=True)

/opt/homebrew/Caskroom/miniforge/base/envs/tf/lib/python3.8/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [121]:
트립=pd.read_csv('trip.csv').drop('Unnamed: 0',axis=1).rename(columns={'0':'text'})
트립

,text
0,전체적으로 사람이 너무 몰려 정신이 없어요. \n음료는 이십분이 다되서 나왔고\n음...
1,Not a value for money so disappointing . We pa...
2,아이들이 많아서 룸으로 된 식당 검색하다 방문 했는데 별로 좋지 않은 기억만 남았....
3,런치먹으러 광원으로 향하는 길에 차가 너무 많이 막히길래 전화해서 3시에서 5분정....
4,"Overprice, Low level Beef, Bad Services, Stres..."
...,...
977,파리 네마리가 쌍으로 날라다니는데 잡을생각도안하시고 문제있더라구요 맛은 달콤하니 ....
978,刺し身とか、シーフードがいろいろ安いとは思いますが、店員さんの愛想は悪いです。店もあまりきれ...
979,관광버스가 열심히 들락거립니다.\n돗대기 시장 저리가라 입니다.식사중에 와서 계산하...
980,제주 2박3일 일정\n마지막 저녁식사 장소\n그러나.\n공항에서부터 구토..\n비행...


In [125]:
text_df=네이버_fil[['text']].append(트립)
text_df_3=네이버_fil_3[['text']]

In [126]:
text_df=text_df.reset_index().drop('index',axis=1)
text_df

,text
0,왜? 맞집일까요?\n빈대떡은 타고 기름이..넘쳐나고\n내가 왜 왔을까요?
1,뚝배기에 김만 모락모락?? 돈앞에 음식이 무너지면 망합니다. 그냥 SNS용 관광식당...
2,고사리 육개장 맛있는데 빈대떡은 다 흩어지고\n앞에 사장님인지 대기표 주시는 어머님...
3,고사리 죽이었음. 한시간 기다려셔 먹은것 치고 후회됨. 이빨없는 어르신들 가시면좋을듯.
4,왜줄서서먹는지.이해1도안감
...,...
9956,파리 네마리가 쌍으로 날라다니는데 잡을생각도안하시고 문제있더라구요 맛은 달콤하니 ....
9957,刺し身とか、シーフードがいろいろ安いとは思いますが、店員さんの愛想は悪いです。店もあまりきれ...
9958,관광버스가 열심히 들락거립니다.\n돗대기 시장 저리가라 입니다.식사중에 와서 계산하...
9959,제주 2박3일 일정\n마지막 저녁식사 장소\n그러나.\n공항에서부터 구토..\n비행...


In [127]:
for i in text_df.index:
    text_df.loc[i,'text']=text_df.loc[i,'text'].replace('\n',' ').replace('...','').replace('ㅋ','').replace('ㅎ','').replace('ㅜ','').replace('ㅠ','').replace('더 보기','')
for i in text_df_3.index:
    text_df_3.loc[i,'text']=text_df_3.loc[i,'text'].replace('\n',' ').replace('...','').replace('ㅋ','').replace('ㅎ','').replace('ㅜ','').replace('ㅠ','').replace('더 보기','')

/opt/homebrew/Caskroom/miniforge/base/envs/tf/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/opt/homebrew/Caskroom/miniforge/base/envs/tf/lib/python3.8/site-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [128]:
text_df['text'] = text_df['text'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", regex=True)
text_df_3['text'] = text_df_3['text'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", regex=True)

/opt/homebrew/Caskroom/miniforge/base/envs/tf/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [129]:
text_df['text'].replace('', np.nan, inplace=True)
text_df_3['text'].replace('', np.nan, inplace=True)

/opt/homebrew/Caskroom/miniforge/base/envs/tf/lib/python3.8/site-packages/pandas/core/generic.py:6610: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [130]:
text_df.dropna(inplace=True)
text_df_3.dropna(inplace=True)
text_df.isna().sum()

/opt/homebrew/Caskroom/miniforge/base/envs/tf/lib/python3.8/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


text    0
dtype: int64

In [137]:
checked_sents=[spell_checker.check(i).checked for i in text_df['text']]
checked_sents_3=[spell_checker.check(i).checked for i in text_df_3['text']]

In [335]:
naver=pd.read_csv('naver_review.csv')
naver

,text,label
0,왜? 맞집일까요?\n빈대떡은 타고 기름이..넘쳐나고\n내가 왜 왔을까요?,0.5
1,친절과 위생은 ㅎㅎ.. 그치만 맛있어요,5.0
2,늘 맛있는 곳,5.0
3,별 다섯개주기엔 대기가 넘 길어요 이정도면 확장하셔요~~~ 포장하면 호딱들고갈수있음...,4.0
4,소문대로 맛나요,5.0
...,...,...
158727,제주하면 뭐다 ? 갈치조림 ~~~~🧡 서비스도 넉넉해서 배터져죽을뻔했어요 ㅠㅠㅠ 담...,5.0
158728,별한개도 아깝습니다. 처음으로 리뷰쓸려고 영수증 리뷰에 가입했습니다.해물뚝배기 먹었...,1.0
158729,성게미역국은 밍밍 \n조림만 그나마 먹을만,3.0
158730,"전복해물뚝배기 먹었는데\n가격은 15,000원 대비\n그냥저냥 먹을만하다",4.0


In [241]:
naver=naver[(naver.label==5.0)|(naver.label==4.5)|(naver.label==4.0)]

In [244]:
naver

,text,label
1,친절과 위생은 ㅎㅎ 그치만 맛있어요,5.0
2,늘 맛있는 곳,5.0
3,별 다섯개주기엔 대기가 넘 길어요 이정도면 확장하셔요 포장하면 호딱들고갈수있음 담부...,4.0
4,소문대로 맛나요,5.0
6,맛있어요,5.0
...,...,...
158725,점심으로 먹고 왔어용 ㅎㅎ 제주 인심을 느낄수있었어용 국물도 진해서 너무 든든해요...,5.0
158726,너무 맛있게 잘 먹고 갑니당 ㅎㅎ 모든 음식이 다 맛있었네요,5.0
158727,제주하면 뭐다 갈치조림 서비스도 넉넉해서 배터져죽을뻔했어요 ㅠㅠㅠ 담엔 부모님 ...,5.0
158730,전복해물뚝배기 먹었는데가격은 원 대비그냥저냥 먹을만하다,4.0


In [ ]:
RandomUnderSampler()

In [234]:
naver.groupby('label').count()

,text
label,
4.0,20160
4.5,13758
5.0,105377


In [243]:
naver['text']=naver['text'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", regex=True)

/opt/homebrew/Caskroom/miniforge/base/envs/tf/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [245]:
naver['text']=naver['text'].replace('\n',' ').replace('ㅋ','').replace('ㅎ','').replace('ㅜ','').replace('ㅠ','')

/opt/homebrew/Caskroom/miniforge/base/envs/tf/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [246]:
for i in naver.index:
    naver.loc[i,'text']=naver.loc[i,'text'].replace('\n',' ').replace('ㅋ','').replace('ㅎ','').replace('ㅜ','').replace('ㅠ','')

/opt/homebrew/Caskroom/miniforge/base/envs/tf/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [247]:
naver['text'].replace('', np.nan, inplace=True)

/opt/homebrew/Caskroom/miniforge/base/envs/tf/lib/python3.8/site-packages/pandas/core/generic.py:6610: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [249]:
naver.dropna(inplace=True)

/opt/homebrew/Caskroom/miniforge/base/envs/tf/lib/python3.8/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [250]:
naver.isna().sum()

text     0
label    0
dtype: int64

In [251]:
naver['label']=naver.label.map({5.0:0,4.0:0,4.5:0})

In [252]:
naver.rename(columns={'text':'리뷰'},inplace=True)

/opt/homebrew/Caskroom/miniforge/base/envs/tf/lib/python3.8/site-packages/pandas/core/frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [255]:
review=pd.read_csv('review.csv')

In [256]:
review.groupby('label').count()

,리뷰
label,
0,56486
1,22479
2,13328


In [262]:
review=review[review.label==1]

In [263]:
review=review.append(naver)

In [264]:
review.groupby('label').count()

,리뷰
label,
0,138221
1,22479


In [265]:
txt=review['리뷰']
label=review['label']

In [274]:
txt=np.array(txt).reshape(-1,1)
label=np.array(label).reshape(-1,1)

In [261]:
from imblearn.under_sampling import RandomUnderSampler

In [275]:
txt, label= RandomUnderSampler(random_state=0).fit_resample(txt, label)

In [312]:
naver=pd.DataFrame(txt,label).reset_index().rename(columns={'index':'label',0:'리뷰'})

In [314]:
review=pd.read_csv('review.csv')
review=review[(review.label==2)]

In [318]:
review=review.append(naver)

In [326]:
checked_sents=[spell_checker.check(i).checked for i in review[review['label']==0]['리뷰']]

In [333]:
print(checked_sents[:10])
print(review[review['label']==0].loc[:10,'리뷰'])

['왜 인기 많은지 알 거 같아요 해삼 먹으려고 했는데 해삼은 일찍 가야 있고 나머진 시비 후에도 먹을 수 있어요', '구성비 굳 서빙  불쾌하여  점 감점', '광어 고등어 농어회 포장해서 먹었는데 저렴하고 맛있었어요 사람들도 엄청 많아요', '월 할 때부터 다녔는데 이제 너무 비싸졌지만 정말 사람 많습니다 그래도 맛은 따라올 수가 없죠 ', '맛있네요 사장님도 친절하고 또 올게요', '친절  맛  위생 내용물 실하고 맛있어요', '비싸서 걱정했는데 가격 값을 분명히 합니다 고등어 냄새가 하나도 안 나네요 맛있게 잘 먹고 갑니다', '육지에서 친척들이 내려와서 심사숙고 끝에 예약해서 찾아갔는데 웨이팅도 적당했고 단체 인원이 식사하기에 괜찮았어요 어르신들 입맛이 워낙 까다로웠는데 다들 만족하셔서 안심했어요 다음에도 또 데려가도 될 만큼 여기 인정합니다 중문 맛 집 제주갈치조림', '여기 진짜 맛있었어요 해물만두전골 처음 먹어봤는데 딱새우가 먹기 힘들었지만 나머지는 정말 맛있었어요', '정말 맛있게 잘 먹었습니다 구성도 좋고 가격 대비 이렇게 나와도 되나 싶을 정도로 퀄리티가 굉장히 좋았습니다']
0     왜 인기 많은지 알거같아요 해삼 먹으려고 했는데 해삼은 일찍가야 있고 나머진 시이후...
1                                  가성비 굳 서빙  불쾌하여  점 감점
2              광어고등어농어회 포장해서 먹었는데 저렴하고 맛있었어요 사람들도 엄청많아요
3     월 할때부터 다녔는데 이제 너무 비싸졌지만 정말 사람많습니다그래도 맛은 따라올수가 없죠 
4                                  맛있네요 사장님도 친절하고 또 올게요
5                                친절  맛  위생 내용물 실하고 맛있어요
6     비싸서 걱정했는데 가격 값을 분명히 합니다 고등어 냄새가 하나도 안나네요 맛있게 잘...
7     육지에서 친척들이 내려와서 심사숙고 끝에 예약해서 찾아갔는데 웨이팅

In [325]:
review.to_csv('re_review.csv',encoding='utf-8',index=False)

In [141]:
checked_df=pd.DataFrame(checked_sents,columns=['리뷰'])
checked_df['label']=1
checked_df3=pd.DataFrame(checked_sents_3,columns=['리뷰'])
checked_df3['label']=2
checked_df3

,리뷰,label
0,특별했던 육개장입니다,2
1,해장국은 진짜 아무 맛이 안 나네요 선지도 너무 적고 무조건 고사리 육개장만 드세요...,2
2,닭개장 같은 맛이에요 한 시간 기다려서 먹을 정도는 아니고 포장하면 분만에 나오니까...,2
3,제주 고사리에 대한 환상으로 갔다 푹 퍼진 육개장에 실망했습니다 제가 뭘 모르고 갔...,2
4,해장국은 맛있는데 밑반찬은 아쉬워요,2
...,...,...
5380,그냥저냥 별루별류,2
5381,맛있는데 푸짐하지는 않아요,2
5382,아치 먹으러 갔어요 딱 거기까지,2
5383,성게알 미역국을 먹었습니다 첨 먹어본 음식이긴 한데 많이 짜고 맛도 잘 모르겠습니다...,2


In [143]:
review=review.append([checked_df,checked_df3])

In [144]:
review

,리뷰,별점,label
0,가보면 내 맘 알지,5점,0
1,일주일 동안 흑돼지 3번 먹었는데 여기가 2위로 괜찮았으면 좋은 점 직원 친절도...,3점,2
2,2번째 방문했어요 ㅎㅎ 고기는 당연히 맛있고 사장님께서 엄청나게 깔끔하신 성격 같아...,5점,0
3,평점 5점이 아닌 게 이상하다 생각이 드는 식당,5점,0
4,제주 갈 때마다 늘 가요,5점,0
...,...,...,...
5380,그냥저냥 별루별류,NaN,2
5381,맛있는데 푸짐하지는 않아요,NaN,2
5382,아치 먹으러 갔어요 딱 거기까지,NaN,2
5383,성게알 미역국을 먹었습니다 첨 먹어본 음식이긴 한데 많이 짜고 맛도 잘 모르겠습니다...,NaN,2


In [145]:
review.to_csv('review.csv',index=False,encoding='utf-8')

In [364]:
rereview=pd.read_csv('re_re_review.csv')

In [366]:
checked_sents=[spell_checker.check(i).checked for i in rereview[rereview.label==0]['리뷰']]

In [367]:
rereview=rereview[(rereview.label==1)|(rereview.label==2)]
for i in checked_sents:
    rereview=rereview.append([{'리뷰':i,'label':0}])


In [368]:
rereview=rereview.reset_index().drop('index',axis=1)

In [370]:
rereview.to_csv('re_re_review.csv',index=False,encoding='utf-8')